In [1]:
dbutils.library.installPyPI("azureml-sdk")

Out[1]: True

In [2]:
from azureml.core import Workspace, Experiment, Datastore, Dataset, Environment

In [3]:
import azureml.core

from azureml.core import Workspace

subscription_id = '0e9bace8-7a81-4922-83b5-d995ff706507'

# Azure Machine Learning resource group NOT the managed resource group
resource_group = 'azureml' 

#Azure Machine Learning workspace name, NOT Azure Databricks workspace
workspace_name = 'ws01ent'  

# Instantiate Azure Machine Learning workspace
ws = Workspace.get(name=workspace_name,
                   subscription_id=subscription_id,
                   resource_group=resource_group)

Performing interactive authentication. Please follow the instructions on the terminal.
To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code ANJ23LT56 to authenticate.
You have logged in. Now let us find all the subscriptions to which you have access...
WARNING - Failed to authenticate to tenant '42b03d0b-d7f2-403e-b764-0dbdcf0505f6' due to error 'Get Token request returned http error: 400 and server response: {"error":"interaction_required","error_description":"AADSTS53003: Access has been blocked by Conditional Access policies. The access policy does not allow token issuance.\r\nTrace ID: 73822736-cc25-47b9-b8c4-da46b3a05500\r\nCorrelation ID: 4cb12338-272e-40df-891c-6cb449a5a39d\r\nTimestamp: 2020-10-26 22:31:24Z","error_codes":[53003],"timestamp":"2020-10-26 22:31:24Z","trace_id":"73822736-cc25-47b9-b8c4-da46b3a05500","correlation_id":"4cb12338-272e-40df-891c-6cb449a5a39d","error_uri":"https://login.microsoftonline.com/error?code=53003","suberror":"message_only"}'.Will continue to look for other tenants to find subscriptions to which you have access
Interactive authentication successfully completed.

In [4]:
spark.conf.set(
  "fs.azure.account.key.adlsdatalakegen6.dfs.core.windows.net",
  dbutils.secrets.get(scope="scope1",key="adlsdatalakegen6"))

In [5]:
from datetime import datetime
foldername = str(datetime.now())
outdf = spark.sql("select * from delta.`abfss://mltraining@adlsdatalakegen6.dfs.core.windows.net/ISDWeatherDelta` where year=2008 and month = 5 and day = 1")
outdf = outdf.dropna(thresh=1)
outdf.write.format("parquet").save(f"abfss://mltraining@adlsdatalakegen6.dfs.core.windows.net/output/{foldername}")


In [6]:
adlsdatalake_ds = Datastore(ws, "adlsdatalakegen6")

In [7]:
file_list = dbutils.fs.ls(f"abfss://mltraining@adlsdatalakegen6.dfs.core.windows.net/output/{foldername}")
file_list = [file[0].split("/")[-1] for file in file_list if ".parquet" in file[0]]
file_list

Out[13]: ['part-00000-tid-1960677883838820715-a018187e-fef6-4e69-bc10-fdad5cd7c0ba-101-1-c000.snappy.parquet',
 'part-00001-tid-1960677883838820715-a018187e-fef6-4e69-bc10-fdad5cd7c0ba-102-1-c000.snappy.parquet',
 'part-00002-tid-1960677883838820715-a018187e-fef6-4e69-bc10-fdad5cd7c0ba-103-1-c000.snappy.parquet',
 'part-00003-tid-1960677883838820715-a018187e-fef6-4e69-bc10-fdad5cd7c0ba-104-1-c000.snappy.parquet']

In [8]:
aml_parquet_file_list = [(adlsdatalake_ds, f"output/{foldername}/"+file) for file in file_list ]
aml_parquet_file_list

Out[14]: [(<azureml.data.azure_data_lake_datastore.AzureDataLakeGen2Datastore at 0x7fa6a80bd210>,
 'output/2020-10-26 22:31:33.977707/part-00000-tid-1960677883838820715-a018187e-fef6-4e69-bc10-fdad5cd7c0ba-101-1-c000.snappy.parquet'),
 (<azureml.data.azure_data_lake_datastore.AzureDataLakeGen2Datastore at 0x7fa6a80bd210>,
 'output/2020-10-26 22:31:33.977707/part-00001-tid-1960677883838820715-a018187e-fef6-4e69-bc10-fdad5cd7c0ba-102-1-c000.snappy.parquet'),
 (<azureml.data.azure_data_lake_datastore.AzureDataLakeGen2Datastore at 0x7fa6a80bd210>,
 'output/2020-10-26 22:31:33.977707/part-00002-tid-1960677883838820715-a018187e-fef6-4e69-bc10-fdad5cd7c0ba-103-1-c000.snappy.parquet'),
 (<azureml.data.azure_data_lake_datastore.AzureDataLakeGen2Datastore at 0x7fa6a80bd210>,
 'output/2020-10-26 22:31:33.977707/part-00003-tid-1960677883838820715-a018187e-fef6-4e69-bc10-fdad5cd7c0ba-104-1-c000.snappy.parquet')]

In [9]:
aml_dataset = Dataset.Tabular.from_parquet_files(aml_parquet_file_list)

In [10]:
aml_dataset = aml_dataset.with_timestamp_columns('datetime')

aml_dataset.register(ws, "ISDWeatherDS", create_new_version= True)

Out[16]: {
 "source": [
 "('adlsdatalakegen6', 'output/2020-10-26 22:31:33.977707/part-00000-tid-1960677883838820715-a018187e-fef6-4e69-bc10-fdad5cd7c0ba-101-1-c000.snappy.parquet')",
 "('adlsdatalakegen6', 'output/2020-10-26 22:31:33.977707/part-00001-tid-1960677883838820715-a018187e-fef6-4e69-bc10-fdad5cd7c0ba-102-1-c000.snappy.parquet')",
 "('adlsdatalakegen6', 'output/2020-10-26 22:31:33.977707/part-00002-tid-1960677883838820715-a018187e-fef6-4e69-bc10-fdad5cd7c0ba-103-1-c000.snappy.parquet')",
 "('adlsdatalakegen6', 'output/2020-10-26 22:31:33.977707/part-00003-tid-1960677883838820715-a018187e-fef6-4e69-bc10-fdad5cd7c0ba-104-1-c000.snappy.parquet')"
 ],
 "definition": [
 "GetDatastoreFiles",
 "ReadParquetFile",
 "DropColumns"
 ],
 "registration": {
 "id": "f3ac3089-a7f4-4a58-b746-0e65a2541502",
 "name": "ISDWeatherDS",
 "version": 6,
 "workspace": "Workspace.create(name='ws01ent', subscription_id='0e9bace8-7a81-4922-83b5-d995ff706507', resource_group='azureml')"
 }
}